In [18]:
from eye_plus import PlaneZone, eye_good_result
import pybaseball as pb
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option("display.max_columns",100)

In [2]:
# uncomment this if you are downloading for the first time# %%time
# statcast = pb.statcast("2019-03-28", "2019-10-01")# statcast.to_pickle("statcast2019.pkl")

In [4]:
def get_edge(df):
    zone = PlaneZone(top=df.sz_top.mean(), bottom=df.sz_bot.mean())
    ext_zone = zone.extended_zone()
    result_df = df.copy()
    result_df["in_zone"] = result_df.apply(lambda x: zone.is_pitch_in_zone(x.plate_x,x.plate_z) ,axis=1)
    result_df["in_ext_zone"] = result_df.apply(lambda x: ext_zone.is_pitch_in_zone(x.plate_x,x.plate_z) ,axis=1)
    result_df = result_df[(~result_df.in_zone) & (result_df.in_ext_zone)]
    result_df["good_eye"] = result_df.apply(eye_good_result, axis=1)
    return result_df

In [5]:
%%time
stat_edge = get_edge(pd.read_pickle("statcast2019.pkl"))

CPU times: user 58.9 s, sys: 7.08 s, total: 1min 5s
Wall time: 1min 42s


In [94]:
eye_2019 = stat_edge.groupby(["batter"]).agg({"batter" : "size", "good_eye" : {"good_results" : "sum", "eye" : "mean"}})
eye_2019

batter     good_eye          
           size good_results       eye
batter                                
282332.0      3          1.0  0.333333
405395.0    493        340.0  0.689655
408234.0    512        336.0  0.656250
425772.0    221        142.0  0.642534
425783.0    657        495.0  0.753425
...         ...          ...       ...
670712.0    148         90.0  0.608108
670950.0     30         22.0  0.733333
670970.0      1          0.0  0.000000
671790.0      4          2.0  0.500000
676606.0     14          8.0  0.571429

[937 rows x 3 columns]

In [96]:
eye_2019.columns =  ["edge_chances", "good_result", "eye"]
eye_2019 = eye_2019.assign(eye_plus= lambda x: x.eye/x.eye.mean() * 100)

In [97]:
eye = eye_2019.reset_index().merge(pb.playerid_reverse_lookup(eye_2019.index), left_on="batter", right_on="key_mlbam")

Gathering player lookup table. This may take a moment.


In [98]:
eye_over_100 = eye.query("edge_chances > 100")

In [99]:
eye_over_100

,batter,edge_chances,good_result,eye,eye_plus,name_last,name_first,key_mlbam,key_retro,key_bbref,key_fangraphs,mlb_played_first,mlb_played_last
1,405395.0,493,340.0,0.689655,105.009255,pujols,albert,405395,pujoa001,pujolal01,1177,2001.0,2019.0
2,408234.0,512,336.0,0.656250,99.922869,cabrera,miguel,408234,cabrm001,cabremi01,1744,2003.0,2019.0
3,425772.0,221,142.0,0.642534,97.834414,mathis,jeff,425772,mathj001,mathije01,3448,2005.0,2019.0
4,425783.0,657,495.0,0.753425,114.719014,choo,shin-soo,425783,choos001,choosh01,3174,2005.0,2019.0
8,425877.0,379,225.0,0.593668,90.393850,molina,yadier,425877,moliy001,molinya01,7007,2004.0,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,669257.0,195,145.0,0.743590,113.221517,smith,will,669257,smitw003,smithwi05,19197,2019.0,2019.0
925,669374.0,313,192.0,0.613419,93.401203,hiura,keston,669374,hiurk001,hiurake01,20003,2019.0,2019.0
929,670032.0,353,253.0,0.716714,109.129306,lopez,nicky,670032,lopen001,lopezni01,19339,2019.0,2019.0
931,670541.0,348,258.0,0.741379,112.884949,alvarez,yordan,670541,alvay001,alvaryo01,19556,2019.0,2019.0


In [131]:
out = eye_over_100[["name_first", "name_last", "edge_chances", "good_result", "eye", "eye_plus"]].assign(rank=lambda x: x["eye_plus"].rank(ascending=False)).sample(50).sort_values("rank")
out

,name_first,name_last,edge_chances,good_result,eye,eye_plus,rank
117,alex,avila,184,153.0,0.831522,126.610343,6.0
525,nick,martini,118,93.0,0.788136,120.004220,21.0
753,matt,thaiss,168,131.0,0.779762,118.729214,30.0
17,curtis,granderson,394,303.0,0.769036,117.095980,41.0
292,roberto,perez,462,352.0,0.761905,116.010224,52.0
528,joe,panik,469,350.0,0.746269,113.629417,65.0
908,jake,rogers,132,97.0,0.734848,111.890543,87.0
58,asdrubal,cabrera,504,369.0,0.732143,111.478575,93.0
709,austin,meadows,579,423.0,0.730570,111.239078,99.0
187,dj,lemahieu,593,433.0,0.730185,111.180540,100.0


In [129]:
import tabulate

def pprint_df(df, drop_index=True):
    return tabulate.tabulate(df.reset_index(drop=drop_index).fillna(0).values, df.columns, tablefmt="pipe")


In [130]:
print(pprint_df(out))

| name_first   | name_last   |   edge_chances |   good_result |      eye |   eye_plus |   rank |
|:-------------|:------------|---------------:|--------------:|---------:|-----------:|-------:|
| luis         | arraez      |            338 |           267 | 0.789941 |   120.279  |   19   |
| mike         | tauchman    |            287 |           220 | 0.766551 |   116.718  |   47   |
| francisco    | cervelli    |            154 |           118 | 0.766234 |   116.669  |   48   |
| austin       | barnes      |            229 |           173 | 0.755459 |   115.029  |   60   |
| matt         | chapman     |            670 |           499 | 0.744776 |   113.402  |   66   |
| yandy        | diaz        |            327 |           241 | 0.737003 |   112.219  |   84   |
| george       | springer    |            547 |           400 | 0.731261 |   111.344  |   95   |
| harrison     | bader       |            379 |           277 | 0.730871 |   111.285  |   96   |
| martin       | prado       |